**CLASSIFIER**

###CUSTOM ARCHITECTURE

In [ ]:
!pip install gradient-descent-the-ultimate-optimizer

In [3]:
# LIBRARIES
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
from torchvision.datasets import ImageFolder
from torchvision import datasets
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report


In [4]:
# GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [5]:
# TRANSFORM
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
# CIFAR-100
train_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

In [7]:
# DATALOADER
batch_size = 128

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [8]:
# 8-CONV LAYER ARCHITECTURE
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(128 * 4 * 4, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 100)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

In [9]:
# MODEL INSTANCE
model = CNN().to(device)


In [10]:
# LOSS / OPTMIMIZER
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# MODEL
num_epochs = 5
train_losses = []

for epoch in range(num_epochs):
    running_loss = 0.0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Loss Update
        running_loss += loss.item()

        if batch_idx % 100 == 99:
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}")
            running_loss = 0.0
    
    train_losses.append(running_loss / len(train_loader))

print("Completed training!")

In [ ]:
model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

accuracy = 100 * total_correct / total_samples
print(f"CUSTOM_MODEL's accuracy on the test data: {accuracy:.2f}%")

# Plot the training loss curve
plt.plot(range(num_epochs), train_losses)
plt.xlabel('Epoch')
plt.ylabel('Training Loss')
plt.title('Training Loss Curve')
plt.show()

In [13]:
# SAVE PARAMS
torch.save(model.state_dict(), 'custom_model.pth')


### FINETUNNING - CUSTOM_MODEL

In [14]:
# LIBRARIES
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

In [15]:
# TRANSFORM
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_file_paths =  '/content/drive/MyDrive/dataset/test'
train_file_paths = '/content/drive/MyDrive/dataset/training'
val_file_paths = '/content/drive/MyDrive/dataset/validation'

test_data = ImageFolder(root=test_file_paths, transform=transform)
train_data = ImageFolder(root=train_file_paths, transform=transform)
valid_data = ImageFolder(root=val_file_paths, transform=transform)

# Create DataLoaders for training and validation
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=32, shuffle=True)

In [16]:
# MODEL
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN()

In [17]:
model.load_state_dict(torch.load('/content/custom_model.pth'))


<All keys matched successfully>

In [ ]:
# REPLACE LAST LAYER 
model.fc_layers = nn.Sequential(
    nn.Linear(128 * 4 * 4, 512),
    nn.ReLU(),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Linear(256, 5)
)
model.to(device)


In [19]:
# LOSS / OPTMIMIZER
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# FINE-TUNNING
num_epochs = 5

train_losses = []
valid_losses = []
best_valid_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    valid_loss = 0.0

    # Training
    for batch_idx, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation
    model.eval()
    with torch.no_grad():
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            valid_loss += loss.item()

    # Calculate average losses
    train_loss /= len(train_loader)
    valid_loss /= len(valid_loader)

    # Store losses
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

    # Print losses
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {train_loss:.4f}, Validation Loss: {valid_loss:.4f}")

    # Check if validation loss improved
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        # Save the model

print("Completed fine-tuning!")

In [ ]:
# LOSS / MATRIX
model.eval()
total_correct = 0
total_samples = 0
predictions = []
test_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()
        predictions.extend(predicted.tolist())
        test_labels.extend(labels.tolist())

accuracy = 100 * total_correct / total_samples
print(f"CUSTOM_MODEL's accuracy on the test data: {accuracy:.2f}%")

# Plot the training and validation loss curves
plt.plot(range(num_epochs), train_losses, label='Training Loss')
plt.plot(range(num_epochs), valid_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Curves')
plt.legend()
plt.show()

# Create the confusion matrix
predictions = np.array(predictions)
test_labels = np.array(test_labels)
cm = confusion_matrix(test_labels, predictions)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# Compute precision, recall, and F1 scores
predictions = np.array(predictions)
test_labels = np.array(test_labels)
report = classification_report(test_loader.dataset.targets, predictions)
print(report)

In [ ]:
torch.save(model.state_dict(), 'custom_model_fined.pth')


### FINETUNNING - MOBILENET_V3_SMALL 

In [ ]:
# LOAD ARCHITECTURE
from torchvision.models import mobilenet_v3_small
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# TRANSFORM
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_file_paths =  '/content/drive/MyDrive/dataset/test'
train_file_paths = '/content/drive/MyDrive/dataset/training'
val_file_paths = '/content/drive/MyDrive/dataset/validation'

test_data = ImageFolder(root=test_file_paths, transform=transform)
train_data = ImageFolder(root=train_file_paths, transform=transform)
valid_data = ImageFolder(root=val_file_paths, transform=transform)

# Create DataLoaders for training and validation
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=32, shuffle=True)

In [ ]:
# MODEL
model = mobilenet_v3_small(pretrained=True)

In [ ]:
num_classes = 5
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, len(train_data.classes))

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [ ]:
# GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
# FINETUNNING
num_epochs = 10

train_losses = []
valid_losses = []
best_valid_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if batch_idx % 100 == 99:
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}")
            running_loss = 0.0
    
    # Calculate average training loss
    avg_train_loss = running_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Validation
    model.eval()
    valid_loss = 0.0
    with torch.no_grad():
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            valid_loss += loss.item()

    # Calculate average validation loss
    avg_valid_loss = valid_loss / len(valid_loader)
    valid_losses.append(avg_valid_loss)

    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_valid_loss:.4f}")

    # Check if validation loss improved
    if avg_valid_loss < best_valid_loss:
        best_valid_loss = avg_valid_loss
        # Save the model

print("Completed fine-tuning!")

In [ ]:
# LOSS / MATRIX
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

model.eval()
total_correct = 0
total_samples = 0
predictions = []
test_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()
        predictions.extend(predicted.tolist())
        test_labels.extend(labels.tolist())

accuracy = 100 * total_correct / total_samples
print(f"MOBILENET_V3_SMALL's accuracy on the test data: {accuracy:.2f}%")

# Plot the training and validation loss curves
plt.plot(range(num_epochs), train_losses, label='Training Loss')
plt.plot(range(num_epochs), valid_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Curves')
plt.legend()
plt.show()

# Create the confusion matrix
predictions = np.array(predictions)
test_labels = np.array(test_labels)
cm = confusion_matrix(test_labels, predictions)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# Compute precision, recall, and F1 scores
predictions = np.array(predictions)
test_labels = np.array(test_labels)
report = classification_report(test_loader.dataset.targets, predictions)
print(report)

In [ ]:
torch.save(model.state_dict(), 'mobilenet_v3_small.pth')


### FINETUNNING - SHUFFLENET_V2_X0_5


In [ ]:
# LOAD ARCHITECTURE
from torchvision.models import shufflenet_v2_x0_5
model = shufflenet_v2_x0_5(pretrained=True)

In [ ]:
num_classes = 5
model.fc = nn.Linear(model.fc.in_features, num_classes)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# FINETUNNING
num_epochs = 20

for epoch in range(num_epochs):
    running_loss = 0.0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Loss Update
        running_loss += loss.item()

        if batch_idx % 100 == 99:
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}")
            running_loss = 0.0
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print("Completed fine-tuning!")

In [ ]:
# LOSS / MATRIX
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

model.eval()
total_correct = 0
total_samples = 0
predictions = []
test_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()
        predictions.extend(predicted.tolist())
        test_labels.extend(labels.tolist())

accuracy = 100 * total_correct / total_samples
print(f"SHUFFLENET_V2_X0_5's accuracy on the test data: {accuracy:.2f}%")

if len(train_losses) > num_epochs:
    train_losses = train_losses[:num_epochs]
elif len(train_losses) < num_epochs:
    train_losses.extend([None] * (num_epochs - len(train_losses)))

if len(valid_losses) > num_epochs:
    valid_losses = valid_losses[:num_epochs]
elif len(valid_losses) < num_epochs:
    valid_losses.extend([None] * (num_epochs - len(valid_losses)))

# Plot the training and validation loss curves
plt.plot(range(num_epochs), train_losses, label='Training Loss')
plt.plot(range(num_epochs), valid_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Curves')
plt.legend()
plt.show()

# Create the confusion matrix
predictions = np.array(predictions)
test_labels = np.array(test_labels)
cm = confusion_matrix(test_labels, predictions)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# Compute precision, recall, and F1 scores
predictions = np.array(predictions)
test_labels = np.array(test_labels)
report = classification_report(test_loader.dataset.targets, predictions)
print(report)

In [ ]:
torch.save(model.state_dict(), 'shufflenet_v2_x0_5.pth')


### FINETUNNING - SQUEEZENET1_0


In [ ]:
from torchvision.models import squeezenet1_0
model = squeezenet1_0(pretrained=True)

In [ ]:
num_classes = 5
model.classifier._modules['1'] = nn.Conv2d(512, num_classes, kernel_size=(1, 1))


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# FINETUNNING
num_epochs = 20

for epoch in range(num_epochs):
    running_loss = 0.0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Loss Update
        running_loss += loss.item()

        if batch_idx % 100 == 99:
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}")
            running_loss = 0.0
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print("Completed fine-tuning!")

In [ ]:
# LOSS / MATRIX
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

model.eval()
total_correct = 0
total_samples = 0
predictions = []
test_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()
        predictions.extend(predicted.tolist())
        test_labels.extend(labels.tolist())

accuracy = 100 * total_correct / total_samples
print(f"SQUEEZENET1_0's accuracy on the test data: {accuracy:.2f}%")

# Plot the training and validation loss curves
plt.plot(range(num_epochs), train_losses, label='Training Loss')
plt.plot(range(num_epochs), valid_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Curves')
plt.legend()
plt.show()

# Create the confusion matrix
predictions = np.array(predictions)
test_labels = np.array(test_labels)
cm = confusion_matrix(test_labels, predictions)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# Compute precision, recall, and F1 scores
predictions = np.array(predictions)
test_labels = np.array(test_labels)
report = classification_report(test_loader.dataset.targets, predictions)
print(report)

In [ ]:
torch.save(model.state_dict(), 'squeezenet1_0.pth')
